In [3]:
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
# from surprise.model_selection.split import train_test_split
from surprise.model_selection import cross_validate, GridSearchCV
import pandas as pd
import numpy as np
import os, io
from surprise import KNNBasic, KNNWithMeans, SVD, SVDpp
from surprise import accuracy

In [4]:
user_items = pd.read_csv("user_item.csv")

In [100]:
games = pd.read_csv("games.csv")

In [5]:
user_items

,item_id,item_name,playtime_forever,playtime_2weeks,items_count,steam_id,user_id,user_url
0,10,Counter-Strike,6,0,277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,0,277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,0,277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
...,...,...,...,...,...,...,...,...
5153204,346330,BrainBread 2,0,0,7,76561198329548331,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153205,373330,All Is Dust,0,0,7,76561198329548331,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153206,388490,One Way To Die: Steam Edition,3,3,7,76561198329548331,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153207,521570,You Have 10 Seconds 2,4,4,7,76561198329548331,76561198329548331,http://steamcommunity.com/profiles/76561198329...


In [24]:
user_items.item_id.nunique()

10978

In [13]:
user_items_played = user_items[user_items.playtime_forever > 0]

In [5]:
user_items_played.item_id.nunique()

10050

In [14]:
user_items_played

,item_id,item_name,playtime_forever,playtime_2weeks,items_count,steam_id,user_id,user_url
0,10,Counter-Strike,6,0,277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
8,300,Day of Defeat: Source,4733,0,277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
9,240,Counter-Strike: Source,1853,0,277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
10,3830,Psychonauts,333,0,277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
...,...,...,...,...,...,...,...,...
5153202,304930,Unturned,677,677,7,76561198329548331,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153203,227940,Heroes & Generals,43,43,7,76561198329548331,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153206,388490,One Way To Die: Steam Edition,3,3,7,76561198329548331,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153207,521570,You Have 10 Seconds 2,4,4,7,76561198329548331,76561198329548331,http://steamcommunity.com/profiles/76561198329...


In [15]:
user_items_played = user_items_played.drop_duplicates(['steam_id','item_id'])
user_items_played

,item_id,item_name,playtime_forever,playtime_2weeks,items_count,steam_id,user_id,user_url
0,10,Counter-Strike,6,0,277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
8,300,Day of Defeat: Source,4733,0,277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
9,240,Counter-Strike: Source,1853,0,277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
10,3830,Psychonauts,333,0,277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
...,...,...,...,...,...,...,...,...
5153202,304930,Unturned,677,677,7,76561198329548331,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153203,227940,Heroes & Generals,43,43,7,76561198329548331,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153206,388490,One Way To Die: Steam Edition,3,3,7,76561198329548331,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153207,521570,You Have 10 Seconds 2,4,4,7,76561198329548331,76561198329548331,http://steamcommunity.com/profiles/76561198329...


In [17]:
user_items_played[['steam_id','item_id']].to_csv("user_item_played_pair.csv", index=False)

In [107]:
user_items_played[user_items_played.steam_id ==76561197970982479]

,item_id,item_name,playtime_forever,playtime_2weeks,items_count,steam_id,user_id,user_url
0,10,Counter-Strike,6,0,277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
8,300,Day of Defeat: Source,4733,0,277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
9,240,Counter-Strike: Source,1853,0,277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
10,3830,Psychonauts,333,0,277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
...,...,...,...,...,...,...,...,...
269,368230,Kingdom: Classic,83,0,277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
272,289130,Endless Legend,593,0,277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
273,268500,XCOM 2,1495,0,277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
275,379720,DOOM,793,0,277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...


In [59]:
game_id_to_name = user_items[["item_id", "item_name"]].copy()

In [60]:
game_id_to_name.drop_duplicates(inplace=True)

In [61]:
game_id_to_name

,item_id,item_name
0,10,Counter-Strike
1,20,Team Fortress Classic
2,30,Day of Defeat
3,40,Deathmatch Classic
4,50,Half-Life: Opposing Force
...,...,...
4913278,375450,NOBUNAGA'S AMBITION: Souzou SengokuRisshiden
5092969,353390,Alienware Steam Machine
5105048,354280,ChaosTower
5119454,433920,Aveyond 4: Shadow Of The Mist


In [101]:
game_id_to_name_2 = games[["id", "app_name"]].copy().rename(columns={'id':'item_id', 'app_name':'item_name'})
game_id_to_name_2.shape

(32132, 2)

In [103]:
all_games = game_id_to_name.append(game_id_to_name_2)

In [104]:
game_id_to_name.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10978 entries, 0 to 5145725
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   item_id    10978 non-null  int64 
 1   item_name  10978 non-null  object
dtypes: int64(1), object(1)
memory usage: 257.3+ KB


In [105]:
all_games.drop_duplicates(inplace=True)

In [83]:
all_games.to_csv("all_games_id_name_pair.csv", index=False)

In [80]:
all_games[all_games.duplicated("item_id", keep=False)].sort_values(by="item_id")

,item_id,item_name
57815,1670.0,Iron Warriors: T-72 Tank Command
32032,1670.0,Iron Warriors: T - 72 Tank Command
237,1930.0,Two Worlds Epic Edition
3288,1930.0,Two Worlds: Epic Edition
32050,2100.0,Dark Messiah of Might & Magic
...,...,...
104832,521450.0,Hunted: One Step too Far
21572,522490.0,Carrie's Order Up!
104835,522490.0,Carrie's Order Up
7262,522660.0,Snooker-online multiplayer snooker game!


In [81]:
all_games.drop_duplicates(subset="item_id", inplace=True)

In [82]:
all_games

,item_id,item_name
0,10.0,Counter-Strike
1,20.0,Team Fortress Classic
2,30.0,Day of Defeat
3,40.0,Deathmatch Classic
4,50.0,Half-Life: Opposing Force
...,...,...
32128,773640.0,Colony On Mars
32129,733530.0,LOGistICAL: South Africa
32130,610660.0,Russian Roads
32131,658870.0,EXIT 2 - Directions


In [ ]:
all_games

In [8]:
duplicate = user_items_played[user_items_played.duplicated(["item_id","item_name"])]

In [9]:
duplicate

,item_id,item_name,playtime_forever,playtime_2weeks,items_count,steam_id,user_id,user_url
280,300,Day of Defeat: Source,220,0,888,76561198035864385,js41637,http://steamcommunity.com/id/js41637
284,240,Counter-Strike: Source,62,0,888,76561198035864385,js41637,http://steamcommunity.com/id/js41637
319,220,Half-Life 2,750,0,888,76561198035864385,js41637,http://steamcommunity.com/id/js41637
320,340,Half-Life 2: Lost Coast,21,0,888,76561198035864385,js41637,http://steamcommunity.com/id/js41637
321,380,Half-Life 2: Episode One,181,0,888,76561198035864385,js41637,http://steamcommunity.com/id/js41637
...,...,...,...,...,...,...,...,...
5153202,304930,Unturned,677,677,7,76561198329548331,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153203,227940,Heroes & Generals,43,43,7,76561198329548331,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153206,388490,One Way To Die: Steam Edition,3,3,7,76561198329548331,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153207,521570,You Have 10 Seconds 2,4,4,7,76561198329548331,76561198329548331,http://steamcommunity.com/profiles/76561198329...


In [10]:
user_items_played.item_id.drop_duplicates().isin(games.id).value_counts()

True     8523
False    1527
Name: item_id, dtype: int64

In [11]:
user_items_played

,item_id,item_name,playtime_forever,playtime_2weeks,items_count,steam_id,user_id,user_url
0,10,Counter-Strike,6,0,277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
8,300,Day of Defeat: Source,4733,0,277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
9,240,Counter-Strike: Source,1853,0,277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
10,3830,Psychonauts,333,0,277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
...,...,...,...,...,...,...,...,...
5153202,304930,Unturned,677,677,7,76561198329548331,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153203,227940,Heroes & Generals,43,43,7,76561198329548331,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153206,388490,One Way To Die: Steam Edition,3,3,7,76561198329548331,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153207,521570,You Have 10 Seconds 2,4,4,7,76561198329548331,76561198329548331,http://steamcommunity.com/profiles/76561198329...


In [26]:
games[games.id == 1670]

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
32032,Strategy First,['Strategy'],Iron Warriors: T - 72 Tank Command,Iron Warriors: T - 72 Tank Command,http://store.steampowered.com/app/1670/Iron_Wa...,2006-07-26,"['Simulation', 'Strategy', 'Tanks', 'Military']",NaN,http://steamcommunity.com/app/1670/reviews/?br...,"['Single-player', 'Multi-player']",4.99,False,1670.0,Strategy First,Mixed,56.0


In [31]:
percent_missing = games.isnull().sum() * 100 / len(games)
missing_value_df = pd.DataFrame({'column_name': games.columns,
                                 'percent_missing': percent_missing})

In [35]:
missing_value_df.sort_values('percent_missing', inplace=True)
missing_value_df

,column_name,percent_missing
url,url,0.000000
early_access,early_access,0.000000
reviews_url,reviews_url,0.003112
id,id,0.003112
app_name,app_name,0.006224
tags,tags,0.507267
specs,specs,2.085084
price,price,4.310211
title,title,6.379734
release_date,release_date,6.432639


In [36]:
games.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'discount_price', 'reviews_url', 'specs', 'price',
       'early_access', 'id', 'developer', 'sentiment', 'metascore'],
      dtype='object')

Useful feature columns: ['publisher', 'genres', 'tags', 'specs', 'price', 'developer']